# Time evolution of harmonic oscillator states in real space
This computational notebook provides the tools to prepare harmonic oscillator states in one dimensional space. One can then shift the position of these states or excite them with a given momentum, to observed the real-time dynamics of the states. This is useful for a variety of aspects, but particularly for coherent states. 

## Split operator method
This notebook solves the dynamics of the Harmonic Oscillator (HO) in a 1D grid in space. This is set-up as a vector of $N_x=100$ points from $x=-6$ to $x=+6$ in HO units. The time evolution is obtained using the Split-Operator Method. Here, one splits the HO hamiltonian $\hat H=\hat K + \hat U$ into kinetic $\hat K = \frac{\hat p^2}{2}$ and potential $\hat U = \frac{\hat x^2}{2}$ terms. The time evolution operator is formally written as $\hat U(\delta t)=e^{-i \hat H \delta t}$. One can also define time evolution operators for the split parts of the hamiltonian, so that $\hat U_p(\delta t)=e^{-i \hat K \delta t}$ is diagonal in momentum space and associated to the kinetic energy, and $\hat U_x(\delta t)=e^{-i \hat U \delta t}$ is diagonal in real space and linked to the potential energy.

The time evolution starts with an eigenstate of the Harmonic Oscillator in real space, $\psi_n(x,t=0)$. One would like to apply the full operator $\hat U(\delta t)$ on this wave function, but this is not straightforward because $\hat K$ cannot be easily expressed in real space. The Split Operator Method consists in using the replacement
$$ \hat U(\delta t) \approx \hat U_x(\delta t/2) \hat U_p(\delta t) \hat U_x(\delta t/2)$$
which is valid for sufficiently small $\delta t$. Note that now each term is independently diagonal in real or momentum space. For instance, the real space part consists simply in multiplying by a phase, 
$$ \hat U_x(\delta t/2) \psi_n(x,t) = e^{-i \frac{x \delta t}{4} }\psi_n(x,t). $$

To apply the operator in momentum space, one can transform the wave function into momentum space, 
$\psi_n(p,t) = FT \{ \psi_n(x,t) \}$. This can be done efficiently by performing a Fast Fourier Transform on the discrete grid of points in real space. This requires some careful manipulation of the indices in the grid in real and momentum space, which is discussed below. With the wavefunction in momentum space, the small time step in the momentum representation is again just a phase multiplication, $$ \hat U_p(\delta t) \psi_n(p,t) = e^{-i \frac{p}{2}\delta t}\psi_n(p,t=0). $$

## Code
### Real space grid
We start by loading the relevant python packages and generating a mesh in real space, which is stored in the xx vector. Note that this is slightly offset from the lower limit and goes through zero.

In [ ]:
# coding: utf-8
##########################################################################
# THIS PYTHON CODE SOLVES THE HARMONIC OSCILLATOR IN REAL SPACE
# USING THE FOURIER GRID HAMILTONIAN METHOD
##########################################################################
import numpy as np
from numpy import linalg as LA
import math
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft
from tqdm import tqdm
from harmonic_oscillator import *

zi=complex(0,1);
pi=math.pi

# NUMBER OF EIGENSTATES TO CONSIDER
N_eigen=3;

# REAL-SPACE MESH DIMENSION AND NUMBER POINTS
xL=6.
Nx=100

# GRID SPACING
delx=2*xL/Nx

# MESH IN X-SPACE - FROM -xL+del x UP TO +xL
xx=np.zeros(Nx)
xx=delx*(np.arange(Nx)-Nx/2.+1)
print(xx)
indelx_origin=int(Nx/2-1)


### Momentum space grid
We generate a momentum space grid. The choice of momenta is entirely dictated by the use of the Fast Fourier Transform algorithm and the transition from a continuous to a discrete Fourier transform.

In [ ]:
# SPACING IN MOMENTUM SPACE
delp=2.*pi/(2.*xL)

# MESH IN MOMENTUM SPACE
pp=np.zeros(Nx)
for i in range(0,Nx) :
    pp[i]=(i-Nx/2)*delp
pp2=np.power(pp,2)

### Initial wavefunctions
We set up a number of $N_\text{eigen}$ eigenvalues of the harmonic oscillator. The <harmonic_oscillator.py> routine in this folder generates these from the standard formula. The eigenvalues are set, and these are plotted for convenience.

In [ ]:
# RENORMALIZE WAVEFUNCTIONS, COMUPTE NORMS FOR CROSS-CHECK AND PLOT
phi=np.zeros([Nx,N_eigen],dtype=complex);
norm=np.zeros(N_eigen);
ener=np.zeros(N_eigen);
for n in range(N_eigen) :
    phi[:,n] = wfho(n,xx)
    norm[n]=np.real( np.sum( np.conj(phi[:,n])*phi[:,n] )*delx )
    ener[n]=n+0.5

    #plot wavefunction as a function of position
    plt.plot(xx,ener[n]*np.ones(Nx),'--',color='grey');
    plt.plot(xx,ener[n]+phi[:,n],'-');

### Time evolution: initial conditioms
Here, we set up the initial condition for the time evolution. The eigenstates are shifted by $p_0$ ($x_0$) units in momentum (real) space. We also set up the vectors that are necessary for the FFT. 

In [ ]:
# The initial wave function is shifted by p_0 units in momentum space, and x_0 units in real space.
p0=1.0
x0=0.0

# ... 0 - Nx-1 -> Range needed in the Fourier transform -> Given by iNx
iNx=np.zeros(Nx)
ii=np.arange(1,Nx/2+1,1)

phift=np.zeros(Nx)
ppift=np.zeros([Nx,N_eigen],dtype=complex);
for nn in range(N_eigen) :
    phift=phi[:,nn]
    # SHIFT IN MOMENTUM SPACE
    phift=np.exp( zi*p0*xx )*phift
    # FOURIER TRANSFORM
    ppp=np.exp(-zi*pi*np.arange(Nx))*fft( np.exp(-zi*pi*np.arange(Nx))*phift )
    # SHIFT IN POSITION SPACE
    ppp_shift=np.exp(-zi*x0*pp)*ppp
    # INVERSE FOURIER TRANSFORM
    ppift[:,nn]=np.exp(+zi*pi*np.arange(Nx))*ifft( np.exp(+zi*pi*np.arange(Nx))*ppp_shift )

### Time evolution
Here, the time evolution is performed for $N_t$ in steps of $\delta t = dt$. This is performed for one specific eigenvalue $n_\text{dyn}$. 

In [ ]:
# START DYNAMICS
Nt=10
dt=0.25
nn_dyn=0
wfx0=ppift[:,nn_dyn]
x2=np.power(xx,2)
for it in tqdm(range (Nt), desc="Loading..."):
    print(it)
    # HALF STEP IN REAL SPACE REPRESENTATION
    wfx1=np.exp(-zi*dt*x2/4)*wfx0
    # FOURIER TRANSFORM
    wfp0=np.exp(-zi*pi*np.arange(Nx))*fft( np.exp(-zi*pi*np.arange(Nx))*wfx1 )
    # ONE STEP IN MOMENTUM SPACE (FACTOR OF 2??)
    wfp1=np.exp(-zi*dt/2*pp2)*wfp0
    # INVERSE FOURIER TRANSFORM
    wfx2=np.exp(+zi*pi*np.arange(Nx))*ifft( np.exp(+zi*pi*np.arange(Nx))*wfp1 )
    # HALF STEP IN REAL SPACE REPRESENTATION
    wfx3=np.exp(-zi*dt*x2/4)*wfx2

    wfx0=wfx3
    plt.plot(xx,np.conj(wfx0)*wfx0)
    plt.savefig("anim1/fig" + str(it).rjust(4,'0') + ".png",transparent=False)
#    plt.cla()
    plt.show()
